In [1]:
!pip install gradio 
!pip install pytesseract



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached orjson-3.10.12-cp312-none-win_amd64.whl.metadata (42 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
   ---------------------------------------- 0.0/57.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/57.2 MB 4.2 MB/s eta 0:00:14
   - -------------------------------------- 2.1/57.2 MB 6.9 MB/s eta 0:00:08
   -- ------------------------------------- 3.9/57.2 MB 7.8 MB/s eta 0:00:07
   --- ------------------------------------ 5.2/57.2

In [2]:
import pandas as pd
import gradio as gr
import numpy as np 
import os 
import matplotlib.pyplot as plt
import requests  # For API calls
import pytesseract  # For OCR
from PIL import Image

In [8]:
!pip install firebase_admin

  Using cached firebase_admin-6.6.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached cachecontrol-0.14.1-py3-none-any.whl.metadata (3.1 kB)
  Using cached google_api_python_client-2.156.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached google_api_core-2.24.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_cloud_firestore-2.19.0-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached msgpack-1.1.0-cp312-cp312-win_amd64.whl.metadata (8.6 kB)
  Using cached googleapis_common_protos-1.66.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached proto_plus-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth-2.37.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached grpcio-1.68.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached grpcio_status-1.68.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached httplib2-0.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wandb 0.16.6 requires protobuf!=4.21.0,<5,>=3.19.0; sys_platform != "linux", but you have protobuf 5.29.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import firebase_admin
from firebase_admin import credentials, firestore, db

# Initialize Firebase Admin SDK
def initialize_firebase():
    # Path to your service account key JSON file
    service_account_path = "database1.json"
    
    # Initialize Firebase Admin
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://database1-517fc-default-rtdb.firebaseio.com/',  # Replace with your Firebase Database URL
    })
    print("Firebase initialized successfully.")
    


# Save data to Realtime Database
def save_data_realtime_db(reference_path, data):
    try:
        ref = db.reference(reference_path)
        ref.set(data)
        print(f"Data saved to Realtime Database: {reference_path}")
    except Exception as e:
        print(f"Error saving to Realtime Database: {e}")
        
        
        
        

# Fetch data from Realtime Database
def fetch_data_realtime_db(reference_path):
    try:
        ref = db.reference(reference_path)
        data = ref.get()
        print(f"Data fetched from Realtime Database: {data}")
        return data
    except Exception as e:
        print(f"Error fetching from Realtime Database: {e}")
        
        

def store_data(item,Quantity): 
    ing_name=item
    ing_quantity=Quantity
    save_data_realtime_db("Data/ingredients/"+str(ing_name), ing_quantity)
    
    
def Fetch_data(item):
    fetched_realtime_db_data = fetch_data_realtime_db("Data/ingredients")
    return fetched_realtime_db_data[item]
    

def Fetch_dataframe():
    fetched_realtime_db_data = fetch_data_realtime_db("Data/ingredients")
    return fetched_realtime_db_data;
    

initialize_firebase()

Firebase initialized successfully.


In [16]:
def operation(action, ingredient, quantity):
    if action == "Add":
        store_data(ingredient,quantity)
        return "Data Added Successfully"
        

    elif action == "Fetch":
        total=Fetch_data(ingredient)
        return total

    

In [17]:
interface = gr.Interface(
    fn=operation,
    inputs=[
        gr.Radio(choices=["Add", "Fetch"], label="Action"),
        gr.Textbox(label="Ingredient"),
        gr.Slider(0, 100, label="Quantity"),
    ],
    outputs="text"
)



In [18]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv
Data saved to Realtime Database: Data/ingredients/Potato
Data fetched from Realtime Database: {'Potato': 39, 'Tomato': 15}


In [12]:
def parse_text(file):
    if file.name.endswith(".txt"):
        return file.read().decode("utf-8")
    else:
        return "Unsupported file format. Only .txt files are allowed."

def parse_image(file):
    image = Image.open(file)
    text = pytesseract.image_to_string(image)
    return text

def save_recipe(text):
    with open("my_fav_recipes.txt", "a") as file:
        file.write(text + "\n")



In [ ]:
interface = gr.Interface(
    fn=operation,
    inputs=[
        gr.File(label="Recipe File (Text or Image)")
    ],
    outputs="text"
)

interface.launch()
